In [1]:
import pandas as pd
import os
import json
from math import isnan
from collections import defaultdict

In [2]:
culture_to_keep = ["VI", "VE", "TP", "T", "S", "PP", "PH", "PE", "PC","PA", "P", "L"]
cols = {
    "date_mutation": str,
    "nature_mutation": str,
    "valeur_fonciere": float,
    "code_commune": str,
    "id_parcelle": str,
    "code_nature_culture": str,
    "nature_culture": str,
    "code_nature_culture_speciale": str,
    "nature_culture_speciale": str,
    "surface_terrain": float,
    "longitude": float,
    "latitude": float,
}
epci = pd.read_csv('epci.csv', usecols=['code_commune', 'code_epci'], dtype=str)
missing_epci = {
    "75": "200054781",
    "13": "200054807",
    "69": "200046977",
}

def load_dvf(year):
    _ = pd.read_csv(
        f'https://files.data.gouv.fr/geo-dvf/latest/csv/{year}/full.csv.gz',
        compression="gzip",
        usecols=list(cols.keys()),
        dtype=cols,
    )
    _ = _.loc[_['code_nature_culture'].isin(culture_to_keep)]
    _['code_departement'] = _['code_commune'].apply(lambda commune: commune[:2] if commune[:2]!="97" else commune[:3])
    _["month"] = _["date_mutation"].str.slice(0, 7)
    merged = pd.merge(
        _,
        epci,
        on='code_commune',
        how="left",
    )
    merged['code_epci'] = merged.apply(
        lambda df: df['code_epci'] if isinstance(df['code_epci'], str) else missing_epci.get(df['code_commune'][:2]),
        axis=1,
    )
    return merged

In [33]:
df = load_dvf(2024)
df

,date_mutation,nature_mutation,valeur_fonciere,code_commune,id_parcelle,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude,code_departement,month,code_epci
0,2024-01-02,Vente,346.5,01076,010760000B0514,P,prés,NaN,NaN,99.0,5.530952,45.952439,01,2024-01,240100883
1,2024-01-03,Vente,10000.0,01103,011030000B1782,S,sols,NaN,NaN,115.0,6.043339,46.282256,01,2024-01,240100750
2,2024-01-08,Vente,249000.0,01203,012030000C1065,S,sols,NaN,NaN,497.0,4.911143,46.247235,01,2024-01,200070555
3,2024-01-09,Vente,20000.0,01185,011851860A0082,P,prés,NaN,NaN,2615.0,5.540564,45.892555,01,2024-01,200042935
4,2024-01-09,Vente,20000.0,01185,011851860A0307,L,landes,NaN,NaN,2496.0,5.542666,45.903499,01,2024-01,200042935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865859,2024-06-28,Vente,2700000.0,75113,75113000BK0002,S,sols,NaN,NaN,131.0,2.368075,48.832248,75,2024-06,200054781
865860,2024-06-28,Vente,2700000.0,75113,75113000BK0002,S,sols,NaN,NaN,131.0,2.368075,48.832248,75,2024-06,200054781
865861,2024-06-28,Vente,2700000.0,75113,75113000BK0002,S,sols,NaN,NaN,131.0,2.368075,48.832248,75,2024-06,200054781
865862,2024-06-28,Vente,2700000.0,75113,75113000BK0002,S,sols,NaN,NaN,131.0,2.368075,48.832248,75,2024-06,200054781


In [7]:
# df[['nature_culture', 'nature_culture_speciale']].value_counts(dropna=False).reset_index().to_csv('natures_speciales.csv', index=False)

In [3]:
def create_stats(df):
    dfs = []
    for echelle in [
        "departement",
        "epci",
#         "commune",
    ]:
        grouped = df.groupby(
            [f"code_{echelle}", "month", "nature_culture"]
        )["nature_culture"]

        nb = grouped.count()
        nb.name = "nb_mutations"
        nb = nb.reset_index()
        nb["echelle"] = echelle
        nb.rename({f"code_{echelle}": "code_geo"}, axis=1, inplace=True)
        dfs.append(nb)
    return pd.concat(dfs, ignore_index=True)

In [7]:
if os.path.isfile('stats.csv'):
    os.remove('stats.csv')
if os.path.isfile('dvf.csv'):
    os.remove('dvf.csv')
for year in range(2019, 2025):
    dvf = load_dvf(year)
    dvf.to_csv(
        "dvf.csv",
        index=False,
        mode="w" if year == 2019 else "a",
        header=year == 2019,
    )
    stats = create_stats(dvf)
    stats.to_csv(
        "stats.csv",
        index=False,
        mode="w" if year == 2019 else "a",
        header=year == 2019,
    )
    print("Done with", year)

Done with 2019
Done with 2020
Done with 2021
Done with 2022
Done with 2023
Done with 2024


In [21]:
# pour stats sur la période
stats = pd.read_csv('stats.csv',dtype={"code_geo": str, "nb_mutations": int})
output = defaultdict(lambda: defaultdict(int))
full_period = stats.groupby(["nature_culture", "code_geo"])["nb_mutations"].sum().reset_index()
for nat in full_period["nature_culture"].unique():
    for code in full_period["code_geo"].unique():
        restr = full_period.loc[(full_period["nature_culture"]==nat)&(full_period["code_geo"]==code), "nb_mutations"]
        if len(restr):
            output[nat][code] += restr.values[0]
for nat in output:
    for code in output[nat]:
        output[nat][code] = int(output[nat][code])
with open("full_period.json", "w") as f:
    json.dump(output, f)

In [24]:
output['prés']

defaultdict(int,
            {'200038990': 778,
             '200042604': 3258,
             '200042729': 2020,
             '200057859': 36,
             '200066389': 7518,
             '200067023': 7332,
             '200067031': 3193,
             '200067205': 14252,
             '200067973': 447,
             '200067999': 581,
             '200068005': 274,
             '200068534': 1010,
             '200069425': 8557,
             '200072676': 1693,
             '243500550': 373,
             '243500667': 1139,
             '246000566': 204,
             '246000582': 281,
             '246000707': 314,
             '246000848': 601,
             '246000913': 331,
             '246100390': 1528,
             '246100663': 1758,
             '247200629': 898,
             '247600729': 639,
             '35': 18537,
             '50': 48702,
             '60': 5740,
             '61': 29074,
             '72': 20067,
             '76': 13708,
             '91': 250,
             '01'

In [23]:
# pour stats mensuelles
stats = pd.read_csv('stats.csv',dtype={"code_geo": str, "nb_mutations": int})
output = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
for nature in stats['nature_culture'].unique():
    for code in stats['code_geo'].unique():
        for month in stats['month'].unique():
#             if month not in output[nature][code]:
#                 output[nature][code][month] = 0
            _ = stats.loc[(stats["code_geo"]==code)&(stats["month"]==month)&(stats["nature_culture"]==nature)]
            if len(_) != 0:
                output[nature][code][month] += _['nb_mutations'].values[0]
# with open("stats.json", "w") as f:
#     json.dump(stats.to_dict(orient="records"), f)
output

In [39]:
stats.loc[(stats['nature_culture'] == "landes") & (stats['code_geo'] == '249740119')]

,code_geo,month,nature_culture,nb_mutations,echelle
35801,249740119,2019-07,landes,2,epci
35804,249740119,2019-08,landes,4,epci
35807,249740119,2019-09,landes,3,epci
35810,249740119,2019-10,landes,4,epci
35813,249740119,2019-11,landes,4,epci
35816,249740119,2019-12,landes,16,epci
105478,249740119,2020-01,landes,4,epci
105481,249740119,2020-02,landes,3,epci
105488,249740119,2020-05,landes,2,epci
105491,249740119,2020-06,landes,4,epci
